In [1]:
!pip3 install nufeb-tools -U --quiet

Traceback (most recent call last):
  File "C:\Users\Jonathan\anaconda3\envs\nufeb\Scripts\pip3-script.py", line 6, in <module>
    from pip._internal.cli.main import main
ModuleNotFoundError: No module named 'pip._internal.cli.main'


In [2]:
import os
from random import uniform
import subprocess
from pathlib import Path
from nufeb_tools import utils,plot
import pandas as pd
from string import Template
import numpy as np
from functools import partial
import time
from skopt import gp_minimize, forest_minimize, dummy_minimize, gbrt_minimize
from skopt.plots import plot_convergence, plot_objective
import matplotlib.pyplot as plt
from scipy import interpolate
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error 
from skopt import dump, load
from skopt.callbacks import CheckpointSaver

test_data = pd.read_excel('/mnt/home/sakkosjo/nufeb-cyano-e-coli/experimental-data/sucrose-OD-IPTG-sweep.xls',sheet_name='data')
from scipy.optimize import curve_fit

Nfeval = 1

def od_func(x):
    """Exponential fit to IPTG vs OD750 experimental data

    Args:
        x (float): IPTG concentration (mM)

    Returns:
        float: Smoothed OD750
    """
    return 0.25482 * np.exp(-x/.06811) + 1.12893

# Smooth OD750 data for fitting
test_data.loc[:,'OD750'] = od_func(test_data.IPTG)

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/home/sakkosjo/nufeb-cyano-e-coli/experimental-data/sucrose-OD-IPTG-sweep.xls'

In [12]:
test_data = pd.read_excel('../experimental-data/sucrose-OD-IPTG-sweep.xls',sheet_name='data')
from scipy.optimize import curve_fit
def od_func(x):
    """Exponential fit to IPTG vs OD750 experimental data

    Args:
        x (float): IPTG concentration (mM)

    Returns:
        float: Smoothed OD750
    """
    return 0.25482 * np.exp(-x/.06811) + 1.12893
def suc_func(x,a,b,c):
    return a*np.exp(-x/b)+c
# Smooth OD750 data for fitting
test_data.loc[:,'OD750'] = od_func(test_data.IPTG)


In [4]:
BASE_DIR = Path(r'C:\Users\Jonathan\Downloads\Runs')
folders = [path for path in BASE_DIR.iterdir() if path.is_dir()]

#Extract output

data = [utils.get_data(directory=str(x)) for x in folders]
Volume = np.prod(data[0].metadata['Dimensions'])
CellNum2OD = Volume*1e6/0.3e-8
SucroseMW = 342.3
dfs = []
for x in data:
    temp = pd.concat([x.ntypes.cyano/CellNum2OD,x.ntypes.step/60/60*x.timestep,x.avg_con.Sucrose.reset_index(drop=True)],axis=1)
    temp.columns=['OD750','Hours','Sucrose']
    temp['IPTG'] = x.metadata['SucRatio']
    dfs.append(temp)
df = pd.concat(dfs,ignore_index=True)
df = df.loc[(df.Hours > 23.8) & (df.Hours < 24)]
df.sort_values(by='IPTG',inplace=True)
df.reset_index(inplace=True)
#save in progress plot


In [14]:
mean=df.groupby('IPTG').mean().reset_index()
mean.head()

,IPTG,index,OD750,Hours,Sucrose
0,0.000,420.4,1.3974,23.888889,2.837252e-20
1,0.005,543.6,1.3872,23.888889,3.278717e-01
2,0.010,2022.0,1.3386,23.888889,6.978445e-01
3,0.020,2902.0,1.3110,23.888889,1.310987e+00
4,0.050,3782.0,1.2540,23.888889,2.438765e+00


In [15]:
std=df.groupby('IPTG').std().reset_index()
std.head()

,IPTG,index,OD750,Hours,Sucrose
0,0.000,313.606122,0.038617,0.0,2.251474e-22
1,0.005,227.782352,0.012834,0.0,5.249244e-03
2,0.010,139.140217,0.035204,0.0,2.431672e-02
3,0.020,139.140217,0.024648,0.0,2.140514e-02
4,0.050,139.140217,0.040583,0.0,5.883013e-02


In [34]:
if len(mean.OD750)==len(test_data.OD750):
    ODerr = np.average((test_data.OD750 - mean.OD750) ** 2, axis=0, weights=test_data.OD750)
else:
    ODerr = (((mean.OD750-test_data.OD750)/test_data.OD750)**2).mean()

if len(mean.Sucrose)==len(test_data.Sucrose):
    SUCerr = np.average((test_data.Sucrose - mean.Sucrose) ** 2, axis=0, weights=test_data.Sucrose*std.Sucrose)
else:
    SUCerr = (((mean.Sucrose-test_data.Sucrose)/test_data.Sucrose)**2).mean()
print(ODerr,SUCerr)

0.0010391861894533156 0.011489403509486996


In [43]:
n=1
c = len(df.IPTG.unique())
df.iloc[n::5,:]

,index,OD750,Hours,Sucrose,IPTG
1,174,1.407,23.888889,2.837218e-20,0.000
6,438,1.404,23.888889,3.360567e-01,0.005
11,2110,1.395,23.888889,7.292293e-01,0.010
16,2990,1.311,23.888889,1.308012e+00,0.020
21,3782,1.242,23.888889,2.398890e+00,0.050
26,1494,1.065,23.888889,2.797041e+00,0.100
31,2286,1.089,23.888889,3.173897e+00,0.200
36,3342,1.101,23.888889,3.209389e+00,0.500
41,1054,1.062,23.888889,3.151028e+00,1.000


In [45]:
ODerr=0
SUCerr=0
for i in range(3):
    temp = df.iloc[i::5,:].reset_index()
    if len(mean.OD750)==len(test_data.OD750):
        ODerr = np.average((test_data.OD750 - temp.OD750) ** 2, axis=0, weights=test_data.OD750)+ODerr
    else:
        ODerr = (((temp.OD750-test_data.OD750)/test_data.OD750)**2).mean()+ODerr

    if len(temp.Sucrose)==len(test_data.Sucrose):
        SUCerr = np.average((test_data.Sucrose - temp.Sucrose) ** 2, axis=0, weights=test_data.Sucrose*std.Sucrose)+SUCerr
    else:
        SUCerr = (((temp.Sucrose-test_data.Sucrose)/test_data.Sucrose)**2).mean()+SUCerr
print(ODerr,SUCerr)

0.007096508894005271 0.040846725080589646


In [ ]:
f, ax = plt.subplots(ncols=2)
ax[0].set_title('Sucrose')
ax[0].plot(test_data.IPTG,test_data.Sucrose,marker='o')
ax[0].plot(df.IPTG,df.Sucrose)
ax[1].set_title('OD750')
ax[1].plot(test_data.IPTG,test_data.OD750,marker='o')
ax[1].plot(df.IPTG,df.OD750)
f.tight_layout()